In [1]:
import pandas as pd
from pulp import *
import numpy as np
import ast
import re


In [35]:
data = pd.read_csv(r'D:\Downloads-HDD\debug.csv')
truck = pd.read_csv(r'D:\Downloads-HDD\NewFinalTrucks.csv')

data['quantity'] = data['quantity'].astype(int)

data = pd.DataFrame([data.iloc[idx]
                       for idx in data.index
                       for _ in range(data.iloc[idx]['quantity'])]).reset_index(drop=True)


data["ItemId"] = data.index + 1
data.head(15)

,name,pid,quantity,length,width,height,volume,weight,ETA,truck_type,origin,destination,ItemId
0,Survey,1,7,3.77,5.00,1.06,19.981000,1257,07-Jul-19,Open,Kolkata,Chennai,1
1,Survey,1,7,3.77,5.00,1.06,19.981000,1257,07-Jul-19,Open,Kolkata,Chennai,2
2,Survey,1,7,3.77,5.00,1.06,19.981000,1257,07-Jul-19,Open,Kolkata,Chennai,3
3,Survey,1,7,3.77,5.00,1.06,19.981000,1257,07-Jul-19,Open,Kolkata,Chennai,4
4,Survey,1,7,3.77,5.00,1.06,19.981000,1257,07-Jul-19,Open,Kolkata,Chennai,5
5,Survey,1,7,3.77,5.00,1.06,19.981000,1257,07-Jul-19,Open,Kolkata,Chennai,6
6,Survey,1,7,3.77,5.00,1.06,19.981000,1257,07-Jul-19,Open,Kolkata,Chennai,7
7,Glass Cutting Tools/Glass Cutters,2,1,7.76,3.39,5.16,135.741024,796,14-Jul-19,Open,Bangalore,Hyderabad,8
8,Bearings & Its Hardware,3,4,4.29,3.04,4.03,52.557648,1474,29-Jul-19,Open,Pune,Surat,9
9,Bearings & Its Hardware,3,4,4.29,3.04,4.03,52.557648,1474,29-Jul-19,Open,Pune,Surat,10


In [38]:
# Item masses, volumes
item_mass = data["weight"].tolist()
item_vol = data["volume"].tolist()

# Mass & volume capacities of trucks
truck_mass = truck["Weight"].tolist()
truck_vol = truck["Volume"].tolist()

# Cost of using each truck
truck_cost = truck["Price"].tolist()
id_series = data["ItemId"].tolist()

In [39]:
def allocator(item_mass,item_vol,truck_mass,truck_vol,truck_cost, id_series):
    n_items = len(item_vol)
    set_items = range(n_items)
    n_trucks = len(truck_cost)
    set_trucks = range(n_trucks)
    

    y = pulp.LpVariable.dicts('truckUsed', set_trucks,
        lowBound=0, upBound=1, cat=LpInteger)

    x = pulp.LpVariable.dicts('itemInTruck', (set_items, set_trucks), 
        lowBound=0, upBound=1, cat=LpInteger)
    
    # Model formulation
    prob = LpProblem("Truck allocation problem", LpMinimize)

    # Objective
    prob += lpSum([truck_cost[i] * y[i] for i in set_trucks])
    # Constraints
    for j in set_items:
        # Every item must be taken in one truck
        prob += lpSum([x[j][i] for i in set_trucks]) == 1

    for i in set_trucks:
        # Respect the mass constraint of trucks
        prob += lpSum([item_mass[j] * x[j][i] for j in set_items]) <= truck_mass[i]*y[i]

        # Respect the volume constraint of trucks
        prob += lpSum([item_vol[j] * x[j][i] for j in set_items]) <= truck_vol[i]*y[i]
    # Ensure y variables have to be set to make use of x variables:
    for j in set_items:
        for i in set_trucks:
            x[j][i] <= y[i]
    
    s = id_series          #id_series

    prob.solve()

    
    x_soln = np.array([[x[i][j].varValue for i in set_items] for j in set_trucks])
    y_soln = np.array([y[i].varValue for i in set_trucks])
    
    lpstatus = LpStatus[prob.status]
    total_cost = value(prob.objective)

    trucks_used = str(sum(([y_soln[i] for i in set_trucks])))        #output3
    a = []
    b = []

    for i in set_items:
        for j in set_trucks:
            if x[i][j].value() == 1:
                a.append(str(j))
                b.append(str(s[i]))


    totalitemvol = sum(item_vol)
    
    totaltruckvol = sum([y[i].value() * truck_vol[i] for i in set_trucks])
    
    trucks_status = []                                                #output4  #sufficient or not
    
    if(totaltruckvol >= totalitemvol):
        trucks_status.append("Trucks are sufficient")
    else:
        trucks_status.append("Trucks are insufficient")
    
    
    zipped = list(zip(a, b))
    packed_items = {} 

    for tr,it in zipped:
        try:
            packed_items[tr].append(it)
        except KeyError:
            packed_items[tr] = [it]
            
    print(packed_items)
    
    output_dict = {}                                                 #output5 #Dictionary with trucks and items 
    for k, v in packed_items.items():
        output_dict[int(k)] = [int(item) for item in v]
    
    
    return output_dict

In [ ]:
output_dict = allocator(item_mass,item_vol,truck_mass,truck_vol,truck_cost,id_series)

working5
